In [1]:
import json

conv_data = json.load(open('./data/대화데이터.json', 'r'))['data']
intent_data = json.load(open('./data/의도데이터.json', 'r'))['data']

In [2]:
import os
import openai


API_KEY = "xxxxxx"

def generate(prompt, temperature):
    openai.api_key = API_KEY
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=prompt,
        temperature=temperature
    )
    return completion.choices[0].message["content"].strip()

# Search Module

In [3]:
# convert conversation documents to text embeddings
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')
corpus = [conv['대화전문'] for conv in conv_data]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [4]:
# convert query to text embedding and do semantic search
def search(query):
    
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=3)

    documents = '```\n' + '\n```\n\n```\n'.join([corpus[idx] for idx in top_results[1]][::-1]) + '\n```'
    
    return documents

# Knowledge Module

In [5]:
# extract and summarize related imformation from documents
def knowledge(conv, docs):
    
    prompt = [
                {"role": "user", "content": f"현재 대화:\n```\n{conv}```\n\n관련 대화:\n{docs}\n\n현재 대화와 관련 있는 정보를 구체적으로 정리해줘."},
            ]
    knowledge_text = generate(prompt, 0.7)
    
    return knowledge_text

# Intent Module

In [6]:
# generate user intent given user question and conversation
def intent(query, conv):
    
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=5)

    few_shot = '\n\n###\n\n'.join([f"대화:\n```\n{intent_data[idx]['대화전문']}\n```\n\n고객질문: {intent_data[idx]['고객질문'].strip('고객: ')}\n\n고객의도: {intent_data[idx]['고객의도']}" for idx in top_results[1]][::-1])
    few_shot += f"\n\n###\n\n대화:\n```\n{conv}\n\n고객질문:{query.strip('고객: ')}\n\n고객의도:"

    prompt = [
                {"role": "user", "content": few_shot},
            ]
    intent_text = generate(prompt, 0.7)
    
    return intent_text

# Dialog Module

In [7]:
conv = """고객: 자동차 등록 어떻게 합니까?
상담사: 자동차 명의 변경하십니까?
고객: 네. 이전 등록하려합니다. 
상담사: 먼저 취득세 고지서를 발급하셔야 합니다. 
고객: 어디서 할 수 있습니까?"""

query = "고객: 어디서 할 수 있습니까?"

In [8]:
# generate answer to user question
def dialog(query, conv):
    
    docs_text = search(query)
    knowledge_text = knowledge(conv, docs_text)
    intent_text = intent(query, conv)
    
    # print(docs_text)
    
    prompt = [
                {"role": "user", "content": f"관련대화:\n{docs_text}\n\n관련정보:\n```\n{knowledge_text}\n```\n\n주어진 정보를 바탕으로 현재 고객의 질문에 답변해줘\n\n고객의도:\n```\n{intent_text}\n```\n\n현재대화:\n```\n{conv}\n상담사:"},
            ]
    dialog_text = generate(prompt, 0.7).split('\n')[0].strip('```').strip()
    
    return dialog_text

# Dialog System

In [9]:
def system():
    
    conv = ''
    
    while True:
        query = input('고객: ')
        if query == 'END':
            break
            
        processed_query = '고객: ' + query
        conv += processed_query
        
        answer = dialog(query, conv)
        processed_answer = '상담사: ' + answer
        print(processed_answer)
        conv += f"\n{processed_answer}\n"
        
    return

In [10]:
system()

고객: 속도위반으로 지로가 날라와서요
상담사: 네. 지로에 운전자 확인과 미확인이란 것은 차 명의자의 범칙금과 차 명의자 외의 운전자의 범칙금이 다르게 부과된다는 것입니다. 자동차 소유주가 운전하셨는지에 따라 차이가 있습니다. 하지만, 운전자 확인 여부와 관련된 벌금의 크기는 자세히 알지 못합니다. 다른 궁금하신 점이 있으신가요?
고객: 할인 받을 방법이 있나요?
상담사: 교통범칙금 할인에 대한 정보는 제가 알고 있는 범위를 벗어나므로 정확한 안내를 드리기 어렵습니다. 교통범칙금 할인에 대한 자세한 내용은 해당 지자체나 교통안전공단 등 관련 기관에 문의하시면 도움이 될 것입니다.
고객: 위반한 운전자가 파출소가서 납부하면 차이가 있나요?
상담사: 네, 차 명의자가 직접 파출소에 방문하여 벌금을 납부하거나 온라인 단속조회 서비스를 이용하여 벌금을 납부하셔도 차 명의자와 운전자 확인 여부에 따라 벌금이 다르게 부과될 수 있습니다. 다른 궁금하신 점이 있으신가요?
고객: 벌금이 어떻게 다르게 부과되나요?
상담사: 차 명의자가 운전하셨을 경우에는 차 명의자의 범칙금이 부과되고, 운전자를 확인할 수 없는 경우에는 미확인 범칙금으로 부과됩니다. 미확인 범칙금은 차 명의자의 범칙금보다 높은 금액으로 부과되며, 범칙금 약정 기간이나 할부금 납부를 할 수 없습니다. 다른 궁금하신 점이 있으신가요?
고객: END
